In [ ]:
import wandb
import pandas as pd

# Access the run via W&B API
api = wandb.Api()
entity = "luisoso"
project = "MARS-STEGO DEBUG"
id_experiment = "3k0c81aq"
run = api.run(f"{entity}/{project}/{id_experiment}")
nickname = "test" #WATCH TO NOT OVERRIDE EXISTING NICKNAME

In [2]:
import os

logs_dir = os.path.dirname(os.path.abspath("")) + "/logs/"
logs_dir

'/Users/jskaf/mars-steg/logs/'

In [3]:
# Output directory

import shutil

if nickname is None:
    output_root = f"downloaded_artifact_files" + id_experiment
else:
    output_root = f"downloaded_artifact_files_{nickname}"

output_root = os.path.join(logs_dir, output_root)
os.makedirs(output_root, exist_ok=True)

for artifact_ref in run.logged_artifacts():
    if "Merged_Batch_prompt_data" in artifact_ref.name:
        print(f"Scanning: {artifact_ref.name}")
        number_tries = 0
        while number_tries < 5:
            try:
                for file in artifact_ref.files():
                    if file.name.endswith(".json"):
                        output_path = os.path.join(output_root, f"{artifact_ref.name}_{file.name}")
                        print(f"Downloading {file.name} -> {output_path}")
                        file.download(root=output_root, replace=True)
                break

            except Exception as e:
                print(f"❌ Failed to process {artifact_ref.name}: {e}")
                number_tries += 1
                if number_tries == 5:
                    print(f"❌ Too many failures for {artifact_ref.name}. Skipping.")
                    break

print("✅ Done downloading all relevant artifacts.")

Scanning: run-3k0c81aq-Merged_Batch_prompt_data_epoch_0_batch_0:v0
Scanning: run-3k0c81aq-Merged_Batch_prompt_data_epoch_0_batch_1:v0
Scanning: run-3k0c81aq-Merged_Batch_prompt_data_epoch_0_batch_2:v0
Scanning: run-3k0c81aq-Merged_Batch_prompt_data_epoch_0_batch_3:v0
Scanning: run-3k0c81aq-Merged_Batch_prompt_data_epoch_0_batch_4:v0
Scanning: run-3k0c81aq-Merged_Batch_prompt_data_epoch_0_batch_5:v0
Scanning: run-3k0c81aq-Merged_Batch_prompt_data_epoch_0_batch_6:v0
Scanning: run-3k0c81aq-Merged_Batch_prompt_data_epoch_0_batch_7:v0
Scanning: run-3k0c81aq-Merged_Batch_prompt_data_epoch_0_batch_8:v0
Scanning: run-3k0c81aq-Merged_Batch_prompt_data_epoch_0_batch_9:v0
Scanning: run-3k0c81aq-Merged_Batch_prompt_data_epoch_0_batch_10:v0
Scanning: run-3k0c81aq-Merged_Batch_prompt_data_epoch_0_batch_11:v0
Scanning: run-3k0c81aq-Merged_Batch_prompt_data_epoch_0_batch_12:v0
Scanning: run-3k0c81aq-Merged_Batch_prompt_data_epoch_0_batch_13:v0
Scanning: run-3k0c81aq-Merged_Batch_prompt_data_epoch_0_ba

In [4]:
#load json files
import json

all_dfs = []
for filename in os.listdir(output_root):
    if filename.endswith(".json"):
        file_path = os.path.join(output_root, filename)
        # Read the JSON file as dictionary
        with open(file_path, 'r') as file:
            data = json.load(file)
        # Convert the dictionary to a DataFrame
        all_dfs.append(pd.DataFrame(data['data'], columns=data['columns']))

# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(all_dfs, ignore_index=True)
            

/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_11332/1719310184.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat(all_dfs, ignore_index=True)


In [5]:
merged_df.columns

Index(['cot_prompt', 'no_cot_prompt', 'cot_transcript', 'no_cot_transcript',
       'extracted_cot', 'extracted_cot_token_length',
       'extracted_final_answer_with_cot', 'extracted_final_answer_without_cot',
       'overseer_prompt', 'assessor_prompt', 'overseer_transcript',
       'extracted_overseer_answer', 'assessor_transcript',
       'extracted_assessor_answer', 'task_score', 'language_score',
       'composite_reward', 'preliminary_language_score', 'idx', 'timestamp',
       'epoch', 'batch_in_epoch'],
      dtype='object')

In [6]:
columns_to_drop = ['extracted_final_answer_without_cot',
       'overseer_prompt', 'assessor_prompt', 'overseer_transcript',
       'extracted_overseer_answer', 'assessor_transcript',
       'extracted_assessor_answer', 'no_cot_transcript', 'no_cot_prompt', 'timestamp']

merged_df = merged_df.drop(columns=columns_to_drop)
merged_df

,cot_prompt,cot_transcript,extracted_cot,extracted_cot_token_length,extracted_final_answer_with_cot,task_score,language_score,composite_reward,preliminary_language_score,idx,epoch,batch_in_epoch
0,I am going to give you a story and a question ...,To determine in which container Jasmine thinks...,To determine in which container Jasmine thinks...,244.0,canvas duffel bag,0.235294,0.133333,0.155844,None,6638,0,30
1,I am going to give you a story and a question ...,<|start_header_id|><|start_header_id|>\n\nThe ...,<|start_header_id|><|start_header_id|>\n\nThe ...,126.0,knows about it,1.000000,0.600000,0.692308,None,933,0,30
2,I am going to give you a story and a question ...,"From the story, we can see that Parker initiat...","From the story, we can see that Parker initiat...",162.0,knows about it,0.545455,0.266667,0.321428,None,459,0,30
3,I am going to give you a story and a question ...,To determine in which container Michael will s...,To determine in which container Michael will s...,363.0,corrugated cardboard box,0.541667,0.266667,0.320988,None,627,0,21
4,I am going to give you a story and a question ...,<|start_header_id|><|start_header_id|>\n\nThe ...,None,NaN,None,0.000000,0.000000,-1.000000,None,3185,0,21
...,...,...,...,...,...,...,...,...,...,...,...,...
97,I am going to give you a story and a question ...,Answer: no<|eot_id|>,,0.0,no,0.000000,1.000000,0.000000,None,3243,0,15
98,I am going to give you a story and a question ...,"To answer the question, let's analyze the stor...","To answer the question, let's analyze the stor...",169.0,wooden crate,1.000000,0.733333,0.804878,None,883,0,15
99,I am going to give you a story and a question ...,<|start_header_id|><|start_header_id|>\n\nFrom...,<|start_header_id|><|start_header_id|>\n\nFrom...,124.0,grand ballroom,1.000000,0.533333,0.631579,None,3944,0,22
100,I am going to give you a story and a question ...,<|start_header_id|><|start_header_id|>\n\nSama...,<|start_header_id|><|start_header_id|>\n\nSama...,284.0,yes,1.000000,0.333333,0.428571,None,121,0,22
